In [2]:
import pandas as pd

In [ ]:
matches = pd.read_csv("soccer-matches.csv")
matches.head()

In [31]:
""" filtered_matches = matches[["Team", "Fixture", "Home Team", "Away Team", "Home Goal", "Away Goal"]]
filtered_matches.to_csv("filtered_soccer_matches.csv", index=False) """

' filtered_matches = matches[["Team", "Fixture", "Home Team", "Away Team", "Home Goal", "Away Goal"]]\nfiltered_matches.to_csv("filtered_soccer_matches.csv", index=False) '

In [ ]:
matches.shape
matches["Team"].value_counts()

In [43]:
matches.dtypes

Team                              object
Date Time (US Eastern)    datetime64[ns]
Fixture                           object
Home Team                         object
Home Goal                        float64
Away Goal                        float64
Away Team                         object
homeAway                          object
foulsCommitted                   float64
yellowCards                      float64
redCards                         float64
offsides                         float64
wonCorners                       float64
saves                            float64
possessionPct                    float64
totalShots                       float64
shotsOnTarget                    float64
shotPct                          float64
penaltyKickGoals                 float64
penaltyKickShots                 float64
accuratePasses                   float64
totalPasses                      float64
passPct                          float64
accurateCrosses                  float64
totalCrosses    

In [42]:
matches["Date Time (US Eastern)"] = pd.to_datetime(matches["Date Time (US Eastern)"])